# InfoCycle — Model Training  

**Author:** Razeen Rahman  

**Description:**  

This notebook trains a ResNet18 deep learning model to classify recyclable materials (e.g., plastic, paper, glass, etc.) using the Garbage Classification dataset.

The data set can be found here: https://www.kaggle.com/datasets/mostafaabla/garbage-classification? 

In [1]:
# imports and device setup
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import time

# device config (supports CUDA, MPS, and CPU)
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA GPU")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using Apple Silicon MPS GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")

print("Selected device:", device)

Using Apple Silicon MPS GPU
Selected device: mps


In [2]:
# dataset path
data_dir = "../datasets/garbage_classification"

train_dir = os.path.join(data_dir)
test_dir = os.path.join(data_dir)

# transforms for training & testing
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


In [3]:
from torch.utils.data import random_split

# set dataset
full_dataset = datasets.ImageFolder(data_dir, transform=train_transforms)
class_names = full_dataset.classes

# 80% for training, 20% for testing
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

# create train and test datasets
splits = random_split(full_dataset, [train_size, test_size])
train_dataset = splits[0]
test_dataset = splits[1]

# remove the transforms from test dataset
test_dataset.dataset.transform = test_transforms

print("Classes:", class_names)
print(f"Train size: {train_size}, Test size: {test_size}")


Classes: ['battery', 'biological', 'brown-glass', 'cardboard', 'clothes', 'green-glass', 'metal', 'paper', 'plastic', 'shoes', 'trash', 'white-glass']
Train size: 12412, Test size: 3103


In [4]:
# 32 images at a time
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [11]:

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# unfreeze layer
for param in model.parameters():
    param.requires_grad = True

# replace final layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(class_names))

# saved model
model.load_state_dict(torch.load("../model/infocycle_v1.pth"))

model = model.to(device)

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-6)


In [ ]:
def train_model(model, train_loader, criterion, optimizer, epochs=2):
    #training mode
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        
        for images, labels in train_loader:
            images = images.to(device) 
            labels = labels.to(device)
            
            optimizer.zero_grad() # reset gradients
            outputs = model(images) # forward pass
            loss = criterion(outputs, labels) # compute loss
            loss.backward() # compute gradients
            optimizer.step() # update weights
            
            running_loss += loss.item()
        
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {running_loss:.4f}")
    
#run training
train_model(model, train_loader, criterion, optimizer, epochs=2)

In [12]:
def evaluate(model, test_loader):
    # evalulation mode
    model.eval()
    
    # counters
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            #get model prediction/output
            outputs = model(images)
            
            # ignore the value
            _, predicted = torch.max(outputs, 1)
            
            # increment counters
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:}%")

# run evaluation
evaluate(model, test_loader)

Test Accuracy: 97.035127296165%


In [10]:
# path
save_path = "../model/infocycle_v1.pth"

# save model
torch.save(model.state_dict(), save_path)

print(f"Model saved to: {save_path}")

Model saved to: ../model/infocycle_v1.pth
